<a href="https://colab.research.google.com/github/indhu68/Intro_To_DL/blob/main/Intro_to_DL_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                                    INTRODUCTION TO DEEP LEARNING

                                            Homework 4

Name: Indhuja Gudluru

Student ID: 801366046

PROBLEM 1:

English To French Translation(Without Attention)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random
from itertools import chain

In [8]:
# Sample dataset of English-French sentence pairs
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]


In [9]:
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = word_to_index = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(english_to_french, char_to_index_Eng, char_to_index_French)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size=len(char_to_index_Eng)
hidden_size=256
output_size=len(char_to_index_French)

# Setting the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Decoder without attention mechanism
class Decoder(nn.Module):
    """Decoder without attention mechanism."""
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_French = Decoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        #Calculate Loss
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')


def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()


            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break
            # Calculate and print loss and accuracy for the evaluation
            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1
            # Optionally, print some examples
            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')
        # Print overall evaluation results
        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)


Epoch 10, Loss: 4.247588947339787
Epoch 20, Loss: 2.9354025886024884
Epoch 30, Loss: 2.232227330816282
Epoch 40, Loss: 1.63039127863257
Epoch 50, Loss: 0.7677924970558466
Epoch 60, Loss: 0.217691672555726
Epoch 70, Loss: 0.10401278132462569
Epoch 80, Loss: 0.06283325965433138
Epoch 90, Loss: 0.03929663051869717
Epoch 100, Loss: 0.026715576520580903
Epoch 110, Loss: 0.02142413196629488
Epoch 120, Loss: 0.017892477508320237
Epoch 130, Loss: 0.015352198403459976
Epoch 140, Loss: 0.01343548882505379
Epoch 150, Loss: 0.011935041607416118
Epoch 160, Loss: 0.010729048553073996
Epoch 170, Loss: 0.009738849659498923
Epoch 180, Loss: 0.008911973571950439
Epoch 190, Loss: 0.008209954390109929
Epoch 200, Loss: 0.007609588016359045
Input: We play music at the concert, Target: Nous jouons de la musique au concert, Predicted: Nous jouons de la musique au concert
Input: She swims in the ocean, Target: Elle nage dans l'océan, Predicted: Elle nage dans l'océan
Input: We dance at the wedding, Target: Nou

PROBLEM 2: With Attention

In [10]:
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling English-French sentence pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(english_to_french, char_to_index_Eng, char_to_index_French)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size_English = len(char_to_index_Eng)
input_size_French = len(char_to_index_French)
hidden_size = 256

# Setting the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize encoder and decoder
class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

class AttnDecoder(nn.Module):
    """Decoder with attention mechanism."""
    def __init__(self, hidden_size, output_size, max_length=12, dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
# Define loss function and optimizer
learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size_English, hidden_size=hidden_size).to(device)
decoder_French = AttnDecoder(hidden_size=hidden_size, output_size=input_size_French).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        #Calculate Loss
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)
        # Perform a single training step and update total loss
        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)




Epoch 10, Loss: 3.50539764579953
Epoch 20, Loss: 2.8767895212470314
Epoch 30, Loss: 2.126536395539453
Epoch 40, Loss: 1.1328513931930742
Epoch 50, Loss: 0.35921001043259015
Epoch 60, Loss: 0.11686782466228099
Epoch 70, Loss: 0.05594712184725582
Epoch 80, Loss: 0.03713106048848058
Epoch 90, Loss: 0.02766682217759622
Epoch 100, Loss: 0.022031334543324604
Epoch 110, Loss: 0.018126565217242253
Epoch 120, Loss: 0.015472108049391538
Epoch 130, Loss: 0.013409706975261013
Epoch 140, Loss: 0.011828434732083136
Epoch 150, Loss: 0.010573457142806986
Epoch 160, Loss: 0.009546020150878081
Epoch 170, Loss: 0.008739724516149797
Epoch 180, Loss: 0.008077020403243476
Epoch 190, Loss: 0.007444308471979798
Epoch 200, Loss: 0.006895485794422423
Input: She speaks French fluently, Target: Elle parle français couramment, Predicted: Elle parle français couramment
Input: She paints a picture, Target: Elle peint un tableau, Predicted: Elle peint un tableau
Input: You are tired, Target: Tu es fatigué, Predicted:

Problem 3: French to English Transl



In [11]:
french_to_english = [(french, english) for (english, french) in english_to_french]

# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling French - English sentence pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(french_to_english, char_to_index_French, char_to_index_Eng)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size_English = len(char_to_index_Eng)
input_size_French = len(char_to_index_French)
hidden_size = 256

class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Decoder without attention mechanism
class Decoder(nn.Module):
    """Decoder without attention mechanism."""
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_French = Decoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)

Epoch 10, Loss: 4.02773656600334
Epoch 20, Loss: 3.168019465568557
Epoch 30, Loss: 2.4189598261904557
Epoch 40, Loss: 1.6702929069322054
Epoch 50, Loss: 0.6961656026583657
Epoch 60, Loss: 0.1982316679695025
Epoch 70, Loss: 0.0807190452358497
Epoch 80, Loss: 0.04704365263530037
Epoch 90, Loss: 0.033726193472724604
Epoch 100, Loss: 0.026183187750547707
Epoch 110, Loss: 0.02132489435045289
Epoch 120, Loss: 0.017948136977786886
Epoch 130, Loss: 0.015469738155898295
Epoch 140, Loss: 0.013573290655549545
Epoch 150, Loss: 0.012077459250159225
Epoch 160, Loss: 0.01087003705940159
Epoch 170, Loss: 0.009875136995244595
Epoch 180, Loss: 0.009041898431752887
Epoch 190, Loss: 0.008334040589430172
Epoch 200, Loss: 0.0077257024115049405
Input: Nous cuisinons le dîner ensemble, Target: We cook dinner together, Predicted: We cook dinner together
Input: Elle enseigne l'anglais à l'école, Target: She teaches English at school, Predicted: She teaches English at school
Input: Il chante dans le chœur, Targe

In [12]:
french_to_english = [(french, english) for (english, french) in english_to_french]

# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 12

char_to_index_Eng = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EnglishFrenchDataset(Dataset):
    """Custom Dataset class for handling French - English sentence pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

English_French_dataset = EnglishFrenchDataset(french_to_english, char_to_index_French, char_to_index_Eng)
dataloader = DataLoader(English_French_dataset, batch_size=1, shuffle=True)

input_size_English = len(char_to_index_Eng)
input_size_French = len(char_to_index_French)
hidden_size = 256

class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# Decoder with attention mechanism
class AttnDecoder(nn.Module):
    """Decoder with attention mechanism."""
    def __init__(self, hidden_size, output_size, max_length=12, dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


learning_rate = 0.01
criterion = nn.NLLLoss()
encoder_English = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_French = Decoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_English.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder_French.parameters(), lr=learning_rate)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

n_epochs = 200

for epoch in range(n_epochs):
    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_English, decoder_French, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if (epoch+10) % 10 == 0:
        print(f'Epoch {epoch+10}, Loss: {total_loss / len(dataloader)}')

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples=5):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()
            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break

            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if i < n_examples:
                predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')

# Perform evaluation
evaluate_and_show_examples(encoder_English, decoder_French, dataloader, criterion)

Epoch 10, Loss: 4.318650618997266
Epoch 20, Loss: 3.2523223112598196
Epoch 30, Loss: 2.3499156616024566
Epoch 40, Loss: 1.6640676664731047
Epoch 50, Loss: 0.6003046738784403
Epoch 60, Loss: 0.1776893020883414
Epoch 70, Loss: 0.07784401276917757
Epoch 80, Loss: 0.046031115745034705
Epoch 90, Loss: 0.03307348831833772
Epoch 100, Loss: 0.025725160853968372
Epoch 110, Loss: 0.020982942144785608
Epoch 120, Loss: 0.017673983714608363
Epoch 130, Loss: 0.015239807041955166
Epoch 140, Loss: 0.013377819542157368
Epoch 150, Loss: 0.011907697757089775
Epoch 160, Loss: 0.01071904493212451
Epoch 170, Loss: 0.00974062375733531
Epoch 180, Loss: 0.008918904794360489
Epoch 190, Loss: 0.008221838090294626
Epoch 200, Loss: 0.007622540140168562
Input: Nous voyageons en train, Target: We travel by train, Predicted: We travel by train
Input: Nous jouons de la musique au concert, Target: We play music at the concert, Predicted: We play music at the concert
Input: Le chat dort, Target: The cat is sleeping, Pre